In [6]:
import os
import psycopg2
import json

# Define your database connection parameters
db_params = {
    "host": "localhost",
    "database": "blockchain",
    "user": "postgres",
    "password": "auth2020"
}

# Specify the block value
block = '6009654'
table_name = 'transaction_header_vote'

# Construct the path to the JSON file
file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

try:
    # Check if the file exists
    if os.path.exists(file_path):
        print(f"Found JSON file: {file_path}")

        # Load JSON data from the file
        with open(file_path, 'r') as json_file:
            json_data = json.load(json_file)

            # Extract other data (type, amount, denom, gas_limit) from your JSON data
            type = json_data["tx"]["body"]["messages"][0]["@type"]
            amount = json_data["tx"]["auth_info"]["fee"]["amount"][0]["amount"]
            denom = json_data["tx"]["auth_info"]["fee"]["amount"][0]["denom"]
            gas_limit = json_data["tx"]["auth_info"]["fee"]["gas_limit"]

            # Connect to the database
            conn = psycopg2.connect(**db_params)
            print("Connected to the database successfully!")

            # Execute the query to retrieve the block_id
            get_block_id_query = f"SELECT block_id FROM block WHERE block = '{block}'"
            with conn.cursor() as cursor:
                cursor.execute(get_block_id_query)
                result = cursor.fetchone()

                if result:
                    block_id = result[0]
                    print(f"Found block_id: {block_id}")

                    # Construct the combined insert query
                    tx_header_insert_query = f"""
                        INSERT INTO {table_name} (block_id, amount, denom, gas_limit)
                        VALUES ('{block_id}', {amount}, '{denom}', {gas_limit});
                    """

                    # Execute the insert query
                    cursor.execute(tx_header_insert_query)
                    conn.commit()  # Commit the transaction

                    print("Data inserted successfully!")
                else:
                    print(f"No block_id found for block {block}")

        # Close the database connection
        conn.close()
    else:
        print(f"JSON file not found: {file_path}")

except psycopg2.Error as e:
    print(f"Error connecting to the database: {e}")



Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/6009654.json


KeyError: 'tx'